# Gradelang

By: Thomas Howard III, Mary Wishart, Brittany Lewis

## Introduction

Grade is an autograding framework for Python which provides more flexibility than traditional Python TestCases by integrating binary testing directly into the language and removing the need for boilerplate code.
Gradelang is a domain specific language which wraps the functionality of the Python package [Grade](https://github.com/thoward27/grade).
It allows graders to create question structures to evaluate student executables, easily access content printed to `stdout` and `stderr`, check exit codes, and more.

By creating Gradelang, we hope it will make it easier for teachers and graders to evaluate student code quickly, easily, and fairly.
We also hope that the ease and flexibility of testing will create more space for creativity in assignment design.

## Implementation

Gradelang is broken up into block structures.

There are four main types of blocks:

- Setup: Statements in the setup block run before every question. Setup will most likely involve statements that create necessary test files or check requirements for questions.
- Teardown: Statements in the teardown block run after every question. This could include operations such as cleaning up created files by deleting them.
- Question: Each question block runs independently from each other question and allows the grader to specify tests and award values associated with those questions.
- Output: This block determines how the results of our program are outputted.

Within those structures we support common arithmetic expressions as well as statements for file creation, variable assignment, arbitrary string, float, and int creation through the hypothesis framework. 
Most importantly tests and be run and tested through the run and assert commands which are supported through the Grade pipeline.

## Challenges

### Independance

One challenge we faced was making each question truly independent.
In order to ensure that one question test failure would not affect any future tests (an important requirement for autograding) we run each question in a separate process, using Python's Multiprocessing library.
In traditional unit tests, infinite loops, calls to exit, or other such severe mechanisms of failure can cause the output to be broken, showing only a subset of the tests provided.
Gradelang has none of these problems.
Since each question is run in a dedicated process, we can kill infinite loops elegantly, even calls to `exit()` within a test will not disrupt the final output.
Gradelang guarentees students will see every test, every time.

To complement the independant nature of each test, we also provide thread-safe storage solutions, strings prefixed with `@/` are treated as paths, and the `@` sybmol replaced by a dedicated workspace for the question currently running.


### Python Bugs

Unfortunately, another issue we ran into was embedded into Python itself.
There exists a bug in Python (Issues [#31935](https://bugs.python.org/issue31935), [#30154](https://bugs.python.org/issue30154), [#37424](https://bugs.python.org/issue37424)), which required us to set a minimum version for Python at 3.7.5.
We recommend using 3.8.0, which is what we built and tested on.

Fortunately, if you do experience any issues with running gradelang, we do provide a Dockerfile preconfigured to work out of the box.

### Hypothesis

In addition to the challenges detailed above, we also had some difficulties getting hypothesis to work with the structure we wanted for our language.
Gradelang provides a convenient wrapper to generate random values at runtime, using `let x be Type`.
To do this, we use Hypothesis, a package designed for property-based testing in Python, which focuses on random value generation and shrinking to minimum failing cases.
Currently, we generate only a single random value for each test, future work will aim to use hypothesis' shrinking functionality by generating many different random values and testing each one in turn.

Please contact the project team with any problems you may encounter.

## Examples

In order to run these examples, we recommend starting with Python 3.8, then installing the dependancies for the project.

```
python -m pip install -r requirements.txt
```

In [1]:
from gradelang.interpreter import interpret

In [2]:
# Basic setup tests
empty = "setup {}"
interpret(empty)

trivial_passing = "setup { assert 1 == 1; }"
interpret(trivial_passing)

trivial_failing = "setup { assert 1 == 0; }"
interpret(trivial_failing)

Grade Results
Grade Results
Grade Results


In [3]:
#Question tests

empty = "question {}"
interpret(empty)

trivial_passing = "question { assert 1 == 1; }"
interpret(trivial_passing)

trivial_failing = "question { assert 0 == 1; }"
interpret(trivial_failing)

testing_output = """
question {
    run "echo hello world";
    assert "hello world" in stdout;
}
"""
interpret(testing_output)

testing_exit_success = """
question {
    run "echo hello world";
    assert exit successful;
}
"""
interpret(testing_exit_success)

testing_create_string="""
    question \"string\"{
        let x be String();
        run "echo", x;
        assert x in stdout;
        award 10;
    }
"""
interpret(testing_create_string)

interpret(testing_exit_success)

name_string = 'question "named" {}'
interpret(name_string)

name_int = 'question 1 {}'
interpret(name_int)

Grade Results
Question 0: 0/0.
Grade Results
Question 0: 0/0.
Grade Results
Question 0: 0/0.
Exception thrown: AssertionError("False was not true! ('==', ('integer', 0), ('integer', 1))")
Traceback (most recent call last):
  File "/home/tom/gradelang/gradelang/interpreter.py", line 63, in worker
    walk(question.body)
  File "/home/tom/gradelang/gradelang/walk.py", line 151, in walk
    return dispatch[action](ast)
  File "/home/tom/gradelang/gradelang/walk.py", line 133, in <lambda>
    'seq': lambda ast: (walk(ast[1]), walk(ast[2])),
  File "/home/tom/gradelang/gradelang/walk.py", line 151, in walk
    return dispatch[action](ast)
  File "/home/tom/gradelang/gradelang/walk.py", line 60, in <lambda>
    'assert': lambda ast: __assert(walk(ast[1]), ast[1]),
  File "/home/tom/gradelang/gradelang/walk.py", line 50, in __assert
    raise AssertionError(f'{cond} was not true! {message}')
AssertionError: False was not true! ('==', ('integer', 0), ('integer', 1))

Grade Results
Question 0: 

In [4]:
#Teardown tests
empty = "teardown {}"
interpret(empty)

trivial_passing = "teardown { assert 1 == 1; }"
interpret(trivial_passing)

trivial_failing = "teardown { assert 0 == 1; }"
interpret(trivial_failing)

Grade Results
Grade Results
Grade Results


In [5]:
#Output tests
empty = "output {}"
interpret(empty)

json = 'output { json; }'
interpret(json)

markdown = 'output { markdown; }'
interpret(markdown)

Grade Results
{
    "tests": []
}
# Results
## Score: 0/0


In [6]:
#Testing programs
empty ="""
setup {}
question {}
teardown {}
output {}
"""
interpret(empty)

setup_failure = """
setup { assert 1 == 0; }
question { assert 1 == 1; }
teardown {}
output {}
"""
interpret(setup_failure)

proposal = """
setup {
    touch "@/temp.txt";
    run "echo";
    assert exit successful;
}

teardown {
    remove "@/temp.txt";
}

output {
    json;
}

question 1 {
    # Run the program, saving output.
    run "echo", "hello world";

    # Now let's run some checks.
    assert exit successful;

    # This checks both stdout and stderr
    assert "hello" in stdout;

    award 10;
}

question 2  {
    run "echo", "hello world";
    assert "goodbye" not in stdout;
    award 10;
    assert "hello" in stdout;
    assert "hello" not in stderr;
    award 10;
}

question 3 {
    let x be Float(minvalue=1);
    run "echo", x;

    # If we want to just look at stdout.
    assert x in stdout;

    String y = "fish";
    run "echo", y;
    assert "fish" in stdout;

    let z be String();
    run "echo", z;
    assert z in stdout;

    let camel be Int(min_value=6);
    run "echo", camel;
    assert camel in stdout;

    award 50;
}
"""
interpret(proposal)

Grade Results
Question 0: 0/0.
Grade Results
Question 0: 0/0.
Exception thrown: AssertionError("False was not true! ('==', ('integer', 1), ('integer', 0))")
Traceback (most recent call last):
  File "/home/tom/gradelang/gradelang/interpreter.py", line 62, in worker
    walk(setup)
  File "/home/tom/gradelang/gradelang/walk.py", line 151, in walk
    return dispatch[action](ast)
  File "/home/tom/gradelang/gradelang/walk.py", line 133, in <lambda>
    'seq': lambda ast: (walk(ast[1]), walk(ast[2])),
  File "/home/tom/gradelang/gradelang/walk.py", line 151, in walk
    return dispatch[action](ast)
  File "/home/tom/gradelang/gradelang/walk.py", line 60, in <lambda>
    'assert': lambda ast: __assert(walk(ast[1]), ast[1]),
  File "/home/tom/gradelang/gradelang/walk.py", line 50, in __assert
    raise AssertionError(f'{cond} was not true! {message}')
AssertionError: False was not true! ('==', ('integer', 1), ('integer', 0))

{
    "tests": [
        {
            "name": "1",
            "

## Conclusions

We successfully created a simple, easy to use language to work with the Grade autograding framework. We believe that this
will make it easier to efficiently grade assignments. We also think our language will be natural and easy to learn
for anyone who has used an autograder in the past.